# Multi-object Photometry

In this section we demonstrate how to perform photometry on multiple objects in an image. We choose a faint group of galaxies and run the photometry steps as described in the [Photometry](./photometry_and_petrosian.ipynb#Photometry) section.

## Loading Example Data

The following data is a cutout of a group of faint galaxies in Abell 2744. The original data was acquired by the [Hubble Frontier Fields](https://frontierfields.org) team via the WFC3 instrument in the `F105W` filter and can be directly downloaded from the [Mikulski Archive for Space Telescopes](https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0/hlsp_frontier_hst_wfc3-60mas_abell2744_f105w_v1.0_drz.fits). 

We first use `astropy`'s ``CCDData`` to load the example data and visualize it through `matplotlib`.

In [ ]:
from astropy.nddata import CCDData

image = CCDData.read('data/abell_2744_group_of_galaxies_f105w.fits.gz')

In [ ]:
# Hidden cell

%matplotlib inline

# Stop Fit Model to Data section warnings
import warnings
warnings.filterwarnings('ignore', append=True)

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'

vmax =  0.01 # Use the image std as max and min of all plots 
vmin = - vmax 

plt.imshow(image.data, vmin=vmin, vmax=vmax)
plt.title("Galaxy Group in Abell 2744 Frontier Field")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

## Make Source Catalog 

We will use the sigma clipped std as a threshold at the segmentation and deblending steps.


In [ ]:
from petrofit.segmentation import make_catalog, plot_segments
from astropy.stats import sigma_clipped_stats

# Compute stats for threshold
image_mean, image_median, image_stddev = sigma_clipped_stats(image.data, sigma=3)

# Define threshold
threshold = image_stddev 

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 5**2


cat, segm, segm_deblend = make_catalog(    
    image.data, 
    threshold=threshold,
    wcs=image.wcs,
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(cat))

## Photometry Loop

We define the list of aperture radii and proceed to the photometry step. In this case, instead of selecting a source, we loop through the source catalog and perform photometry on each object. After constructing the photometry we create a ` Petrosian` object for the source. We save the `Petrosian` in a python dictionary (`petrosian_properties`) for later use.

In [ ]:
from petrofit.photometry import source_photometry
from petrofit.petrosian import Petrosian, PetrosianCorrection
from petrofit.photometry import make_radius_list

max_pix=35

r_list = make_radius_list(
    max_pix=max_pix, # Max pixel to go up to
    n=max_pix # the number of radii to produce 
)


petrosian_properties = {}

for idx, source in enumerate(cat):
    
    # Photomerty 
    flux_arr, area_arr, error_arr = source_photometry(

        # Inputs 
        source, # Source (`photutils.segmentation.catalog.SourceCatalog`)
        image.data, # Image as 2D array 
        segm_deblend, # Deblended segmentation map of image
        r_list, # list of aperture radii  

        # Options 
        cutout_size=max(r_list)*2, # Cutout out size, set to double the max radius  
        bkg_sub=True, # Subtract background  
        sigma=3, sigma_type='clip', # Fit a 2D plane to pixels within 3 sigma of the mean
        plot=False, vmax=vmax, vmin=vmin, # Show plot with max and min defined above
    )
    plt.show()
    
    p = Petrosian(r_list, area_arr, flux_arr)
    
    petrosian_properties[source] = p
    
print("Completed for {} Sources".format(len(petrosian_properties)))

## Compute Total Mags 

We compute the total magnitudes of the sources by looping through their ` Petrosian` objects. We use the ` flux_to_abmag` function to convert the total flux to AB mags of each object.

In [ ]:
import numpy as np 
from petrofit.photometry import flux_to_abmag

mag_list = []

for source in petrosian_properties:
    
    # Get Petrosian
    p = petrosian_properties[source]
    
    # Compute HST Flux -> mags for total_flux
    mag = flux_to_abmag(p.total_flux, image.header)
    
    # Add to mag list
    mag_list.append(mag)

# Convert mag_list to array 
mag_list = np.array(mag_list)

mag_list

## Photometry Catalog

We construct and save a photometry catalog with the magnitudes we computed. To construct the table, we first use the `SourceCatalog.to_table()` function that returns an `astropy` `table`. This will include important info about each source. We then add a new column (`MAG_F105W`) with the total mags we computed. 

In [ ]:
# Segmentation catalog to astropy table.
photo_cat = cat.to_table()

# Add new column with mags.
photo_cat.add_column(mag_list, index=4, name='MAG_F105W')

# Save to file.
photo_cat.write('temp/example_photo_cat.csv', overwrite=True)

photo_cat

# Simultaneous Fitting

In this section, we explore how to make compound models that can be used to describe multiple objects in an image. We use the same dataset as the [Multi-object Photometry section](#Multi-object-Photometry) to fit the nine faint sources with Sersic profiles. 

## Make Individual Models

To do this we loop through the sources and construct `astropy` ` Sersic2D` models for source as described in the [AstroPy Sersic Model sction](./fitting.ipynb#AstroPy-Sersic-Model). We also make initial guesses of the paramters as described in the [Estimating Sersic Parameters](./fitting.ipynb#Estimating-Sersic-Parameters) section. At the end of each iteration, we add the newly constructed model in a list `model_list`.

In [ ]:
from petrofit.segmentation import (get_source_ellip, get_source_elong, get_source_position, 
                                   get_source_theta, get_amplitude_at_r)
from photutils.isophote import EllipseGeometry, Ellipse
from astropy.modeling import models 

# AstroPy Model List
model_list = []

# For each source
for source in list(petrosian_properties.keys()):
    
    # Get Petrosian
    p = petrosian_properties[source]
    
    # Estimate center 
    position = get_source_position(source)    
    x_0, y_0 = position
    
    # Estimate shape 
    elong = get_source_elong(source)
    ellip = get_source_ellip(source)
    theta = get_source_theta(source)
    
    # Estimate Sersic index
    n = 1
    
    # Estimate r_half_light
    r_eff = p.r_half_light
    
    # Estimate amplitude
    amplitude = get_amplitude_at_r(r_eff, image, x_0, y_0, ellip, theta)
    
    # Allow for 4 pixel center slack 
    center_slack = 4
    
    # Make astropy model
    sersic_model = models.Sersic2D(

            amplitude=amplitude,
            r_eff=r_eff,
            n=n,
            x_0=x_0,
            y_0=y_0,
            ellip=ellip, 
            theta=theta,

            bounds = {
                'amplitude': (0., None),
                'r_eff': (0, None),
                'n': (0, 10),
                'ellip': (0, 1),
                'theta': (-2*np.pi, 2*np.pi),
                'x_0': (x_0 - center_slack/2, x_0 + center_slack/2),
                'y_0': (y_0 - center_slack/2, y_0 + center_slack/2),
            },
    )
    
    # Add to model list 
    model_list.append(sersic_model)
    
    # Over-plot Petrosian radii 
    p.imshow(position=position, elong=elong, theta=theta, lw=1.25)

# Plot image of sources 
plt.imshow(image.data, vmax=vmax, vmin=vmin)
plt.show()

## Make Compound Model

To make a single compound model that represents all the sources of interest, we add up all the models. `astropy` models can be added like numbers or arrays, so we convert the model list to a `numpy` array and sum it. 


In [ ]:
compound_model = np.array(model_list).sum()

compound_model

## Make PSFConvolvedImageModel

Now that we have a single model that represents all the sources, we can create a `PSFConvolvedImageModel` with the appropriate parameters. We load a PSF as described in the [PSF section](./fitting.ipynb#Estimating-Sersic-Parameters) of the fitting documentation. We then wrap the compound model and PSF using `PSFConvolvedImageModel`. We specify an oversampling factor 4 to account for poor CCD sampling.

**Load and Normalize PSF**


In [ ]:
from astropy.io import fits

PSF = fits.getdata('data/f105w_psf.fits.gz')
PSF = PSF / PSF.sum()

plt.imshow(PSF, vmin=0, vmax=PSF.std()/10)

**PSFConvolvedImageModel**

In [ ]:
from petrofit.modeling import PSFConvolvedImageModel

psf_sersic_model = PSFConvolvedImageModel(compound_model, psf=PSF, oversample=4)

psf_sersic_model.fixed['psf_pa'] = True

## Fit Model to Data

We fit the compound model using a Levenberg-Marquardt algorithm and save the returned optimized copy of the fitted model in  `fitted_model`. Since this the compound model is composed of many parameters, we may see `astropy` warnings when the fitter explores parameters that cause issues, such as division by zero.

In [ ]:
%%time

from petrofit.modeling import fit_model

fitted_model, _ = fit_model(
    image.data, psf_sersic_model,
    maxiter=10000,
    epsilon=1.4901161193847656e-08,
    acc=1e-09,
)

## Generate Model Image

To generate a model image we use the `model_to_image` utility function. This function allows us to define the center of the model image and the side length of the image.

In [ ]:
from petrofit.modeling import model_to_image

# Make Model Image
# ----------------

# Set the size of the model image equal to the original image
full_fitted_image_size = image.data.shape[0]

# Center the model image at the center of the original image
# so the two images cover the same window
full_fitted_image_center = full_fitted_image_size // 2

# Generate a model image from the model
fitted_model_image = model_to_image(
    fitted_model,
    full_fitted_image_size, 
)

# Plot Model Image
# ----------------

fig, ax = plt.subplots(2, 2)

# Plot raw data
im = ax[0, 0].imshow(image.data, vmin=vmin, vmax=vmax)
ax[0, 0].set_title("Hubble F105W Image")
ax[0, 0].set_xlabel("Pixels")
ax[0, 0].set_ylabel("Pixels")
#ax[0, 0].axis('off')

# Plot Petrosian radii
plt.sca(ax[0, 1])
for i, source in enumerate(petrosian_properties):
    p = petrosian_properties[source]
    
    position = get_source_position(source)    
    x_0, y_0 = position
    
    elong = get_source_elong(source)
    ellip = get_source_ellip(source)
    theta = get_source_theta(source)
    
    p.imshow(position=position, elong=elong, theta=theta, lw=1.25)
    if i == 0:
        plt.legend()
ax[0, 1].imshow(image.data, vmin=vmin, vmax=vmax)
ax[0, 1].set_title("Petrosian Radii")
ax[0, 1].set_xlabel("Pixels")
ax[0, 1].set_ylabel("Pixels")
# ax[0, 1].axis('off')

# Plot Model Image
ax[1, 0].imshow(fitted_model_image, vmin=vmin, vmax=vmax)
ax[1, 0].set_title("Simultaneously Fitted Sersic Models")
ax[1, 0].set_xlabel("Pixels")
ax[1, 0].set_ylabel("Pixels")
# ax[1, 0].axis('off')

# Plot Residual
ax[1, 1].imshow(image.data - fitted_model_image, vmin=vmin, vmax=vmax)
ax[1, 1].set_title("Residual")
ax[1, 1].set_xlabel("Pixels")
ax[1, 1].set_ylabel("Pixels")
# ax[1, 1].axis('off')

plt.show()

## Analysis of Background

We can now create a background image using the residual and perform some statistical  analysis on it.

In [ ]:
# Define background image
background_image = image.data - fitted_model_image

# Compute stats
# -------------

noise_mean = background_image.mean()
noise_sigma = background_image.std()
noise_3_sigma = noise_sigma * 3.
noise_8_sigma = noise_sigma * 8.

print("noise_mean = {}".format(noise_mean))
print("noise_sigma = {}".format(noise_sigma))
print("noise_3_sigma = {}".format(noise_3_sigma))

# Plots
# -----

plt.imshow(image.data - fitted_model_image, vmin=vmin, vmax=vmax)
plt.title('Residual Image')
plt.show()

n, bins, patches = plt.hist(background_image.flatten(), bins=35, align='left', 
                            color='black', label="Binned Residual Image Pixel Values")
plt.plot(bins[:-1], n, c='r', linewidth=3)
plt.axvline(image.data.mean(), label="Raw Input Image Mean", c='g',linestyle="--")
plt.axvline(noise_mean, label="Residual Image Mean", linestyle="--")


plt.xlabel('Flux Bins [{}]'.format(str(image.unit)))
plt.ylabel('Count')
plt.title('Residual Image Pixel Value Histogram')
plt.legend()

plt.show()



Now we see the residual image mean is near that is near the mean of the noise distribution, we can make a segmentation map using the residual image 3-sigma as the detection threshold. Notice how some of the sources we were able to fit were below the 3-sigma estimate of the background (residual image). 

In [ ]:
new_threshold = noise_3_sigma

new_cat, new_segm, new_segm_deblend = make_catalog(    
    image.data, 
    threshold=new_threshold,
    wcs=image.wcs,
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(new_cat))